In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%load_ext blackcellmagic

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd

In [5]:
import extract
import preprocess as prep
import feature_engineering as fe
import imputations as imp
import evaluate as eval_

In [6]:
import sys
sys.path.insert(0,'/home/shared/utils')

## Training

In [7]:
loans = extract.fetch_funded_mature_loans("'2018-01-01'", "'2020-04-19'")

In [8]:
loans = prep.preprocess_iloans(loans)

In [9]:
loans = fe.fe_iloans(loans)

In [10]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21568 entries, 0 to 21567
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   loan_id             21568 non-null  object        
 1   LoanCount           21568 non-null  int64         
 2   OriginationDate     21568 non-null  datetime64[ns]
 3   Campaign            21520 non-null  object        
 4   MonthlyGrossIncome  21568 non-null  float64       
 5   DateOfBirth         21568 non-null  datetime64[ns]
 6   IsFirstDefault      21568 non-null  bool          
 7   Age                 21568 non-null  int64         
 8   LeadProvider        21348 non-null  object        
 9   Reloan              21568 non-null  bool          
dtypes: bool(2), datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 1.4+ MB


In [11]:
access_count = extract.get_esign_time_diff("'2018-01-01'", "'2020-04-19'")

In [12]:
access_count = prep.preprocess_esign(access_count)

In [13]:
access_count.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21568 entries, 0 to 21567
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   loan_id      21568 non-null  object 
 1   AccessCount  21520 non-null  float64
dtypes: float64(1), object(1)
memory usage: 505.5+ KB


In [14]:
bank_reports = extract.fetch_bank_reports("'2018-01-01'", "'2020-04-19'")

In [15]:
bank_reports = prep.preprocess_bank_reports(bank_reports)

In [16]:
bank_reports = fe.fe_bank_reports(bank_reports)

In [17]:
bank_reports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15954 entries, 0 to 15953
Data columns (total 45 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_id                     15954 non-null  object        
 1   BankReportData              15954 non-null  object        
 2   ReportTimeAdded             15954 non-null  datetime64[ns]
 3   primary_account             15954 non-null  object        
 4   txn_days_count              15954 non-null  object        
 5   LenderAmountDeb             15954 non-null  float64       
 6   LenderCountCred             15954 non-null  float64       
 7   LenderAmountCred30          15954 non-null  float64       
 8   LenderCountDeb              15954 non-null  float64       
 9   LenderAmountDeb30           15954 non-null  float64       
 10  LenderCountCred30           15954 non-null  float64       
 11  LenderCountDeb30            15954 non-null  float64   

In [18]:
bank_app = extract.fetch_bank_app_loans("'2018-01-01'", "'2020-04-19'")

In [19]:
bank_app = prep.preprocess_bank_app(bank_app)

In [20]:
bank_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82794 entries, 0 to 82793
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_id                     82794 non-null  object 
 1   bank_app_decision           80359 non-null  object 
 2   reasons_for_decision        79826 non-null  object 
 3   entered_date                82794 non-null  object 
 4   dti                         28755 non-null  float64
 5   in1_is_direct_deposite      59272 non-null  object 
 6   pay_day_test_result_amount  69671 non-null  float64
 7   is_pds_history_found        45780 non-null  object 
dtypes: float64(2), object(6)
memory usage: 5.1+ MB


In [21]:
bank_reports_cols = ['loan_id','LenderAmountDeb', 'LenderCountCred', 'LenderAmountCred30', 'LenderCountDeb',
       'LenderAmountDeb30', 'LenderCountCred30', 'LenderCountDeb30',
       'LenderAmountCred', 'UniqLenderCount']

In [22]:
learning = pd.merge(loans, access_count, how = 'left',on = 'loan_id')

In [23]:
learning.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21568 entries, 0 to 21567
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   loan_id             21568 non-null  object        
 1   LoanCount           21568 non-null  int64         
 2   OriginationDate     21568 non-null  datetime64[ns]
 3   Campaign            21520 non-null  object        
 4   MonthlyGrossIncome  21568 non-null  float64       
 5   DateOfBirth         21568 non-null  datetime64[ns]
 6   IsFirstDefault      21568 non-null  bool          
 7   Age                 21568 non-null  int64         
 8   LeadProvider        21348 non-null  object        
 9   Reloan              21568 non-null  bool          
 10  AccessCount         21520 non-null  float64       
dtypes: bool(2), datetime64[ns](2), float64(2), int64(2), object(3)
memory usage: 1.7+ MB


In [24]:
learning = pd.merge(learning, bank_reports, how = 'left',on = 'loan_id')

In [25]:
learning.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21568 entries, 0 to 21567
Data columns (total 55 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_id                     21568 non-null  object        
 1   LoanCount                   21568 non-null  int64         
 2   OriginationDate             21568 non-null  datetime64[ns]
 3   Campaign                    21520 non-null  object        
 4   MonthlyGrossIncome          21568 non-null  float64       
 5   DateOfBirth                 21568 non-null  datetime64[ns]
 6   IsFirstDefault              21568 non-null  bool          
 7   Age                         21568 non-null  int64         
 8   LeadProvider                21348 non-null  object        
 9   Reloan                      21568 non-null  bool          
 10  AccessCount                 21520 non-null  float64       
 11  BankReportData              15954 non-null  object    

In [26]:
learning = pd.merge(learning, bank_app, how = 'left', on = 'loan_id')

In [27]:
learning.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21568 entries, 0 to 21567
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_id                     21568 non-null  object        
 1   LoanCount                   21568 non-null  int64         
 2   OriginationDate             21568 non-null  datetime64[ns]
 3   Campaign                    21520 non-null  object        
 4   MonthlyGrossIncome          21568 non-null  float64       
 5   DateOfBirth                 21568 non-null  datetime64[ns]
 6   IsFirstDefault              21568 non-null  bool          
 7   Age                         21568 non-null  int64         
 8   LeadProvider                21348 non-null  object        
 9   Reloan                      21568 non-null  bool          
 10  AccessCount                 21520 non-null  float64       
 11  BankReportData              15954 non-null  object    

In [28]:
BV_status_list = ['Bank Validation Uncertain', 'Bank Validation Approved']

In [29]:
learning = learning[learning['bank_app_decision'].isin(BV_status_list)]

In [30]:
learning.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7127 entries, 1 to 21566
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_id                     7127 non-null   object        
 1   LoanCount                   7127 non-null   int64         
 2   OriginationDate             7127 non-null   datetime64[ns]
 3   Campaign                    7122 non-null   object        
 4   MonthlyGrossIncome          7127 non-null   float64       
 5   DateOfBirth                 7127 non-null   datetime64[ns]
 6   IsFirstDefault              7127 non-null   bool          
 7   Age                         7127 non-null   int64         
 8   LeadProvider                7075 non-null   object        
 9   Reloan                      7127 non-null   bool          
 10  AccessCount                 7122 non-null   float64       
 11  BankReportData              5619 non-null   object     

In [31]:
learning = learning.loc[learning['primary_account'].notnull(), :]

In [32]:
learning.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5619 entries, 1 to 21565
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_id                     5619 non-null   object        
 1   LoanCount                   5619 non-null   int64         
 2   OriginationDate             5619 non-null   datetime64[ns]
 3   Campaign                    5617 non-null   object        
 4   MonthlyGrossIncome          5619 non-null   float64       
 5   DateOfBirth                 5619 non-null   datetime64[ns]
 6   IsFirstDefault              5619 non-null   bool          
 7   Age                         5619 non-null   int64         
 8   LeadProvider                5585 non-null   object        
 9   Reloan                      5619 non-null   bool          
 10  AccessCount                 5617 non-null   float64       
 11  BankReportData              5619 non-null   object     

In [33]:
training = learning[(learning['OriginationDate'] >= '2018-01-01') & (learning['OriginationDate'] <= '2019-12-31')]

In [34]:
evaluation = learning[(learning['OriginationDate'] >= '2020-01-01') & (learning['OriginationDate'] <= '2020-04-19')]

In [35]:
training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4527 entries, 1 to 21561
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_id                     4527 non-null   object        
 1   LoanCount                   4527 non-null   int64         
 2   OriginationDate             4527 non-null   datetime64[ns]
 3   Campaign                    4526 non-null   object        
 4   MonthlyGrossIncome          4527 non-null   float64       
 5   DateOfBirth                 4527 non-null   datetime64[ns]
 6   IsFirstDefault              4527 non-null   bool          
 7   Age                         4527 non-null   int64         
 8   LeadProvider                4513 non-null   object        
 9   Reloan                      4527 non-null   bool          
 10  AccessCount                 4526 non-null   float64       
 11  BankReportData              4527 non-null   object     

In [36]:
evaluation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1092 entries, 21 to 21565
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_id                     1092 non-null   object        
 1   LoanCount                   1092 non-null   int64         
 2   OriginationDate             1092 non-null   datetime64[ns]
 3   Campaign                    1091 non-null   object        
 4   MonthlyGrossIncome          1092 non-null   float64       
 5   DateOfBirth                 1092 non-null   datetime64[ns]
 6   IsFirstDefault              1092 non-null   bool          
 7   Age                         1092 non-null   int64         
 8   LeadProvider                1072 non-null   object        
 9   Reloan                      1092 non-null   bool          
 10  AccessCount                 1091 non-null   float64       
 11  BankReportData              1092 non-null   object    

In [37]:
training, imp_acc_count, imp_dti, imp_pay_day = imp.impute_learning(training)

In [38]:
training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4527 entries, 1 to 21561
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_id                     4527 non-null   object        
 1   LoanCount                   4527 non-null   int64         
 2   OriginationDate             4527 non-null   datetime64[ns]
 3   Campaign                    4526 non-null   object        
 4   MonthlyGrossIncome          4527 non-null   float64       
 5   DateOfBirth                 4527 non-null   datetime64[ns]
 6   IsFirstDefault              4527 non-null   bool          
 7   Age                         4527 non-null   int64         
 8   LeadProvider                4527 non-null   object        
 9   Reloan                      4527 non-null   bool          
 10  AccessCount                 4527 non-null   float64       
 11  BankReportData              4527 non-null   object     

In [39]:
imp_acc_count.statistics_, imp_dti.statistics_, imp_pay_day.statistics_

(array([7.]), array([16.]), array([0.]))

In [48]:
evaluation[['AccessCount']] = imp_acc_count.transform(evaluation[['AccessCount']])

In [49]:
evaluation[['dti']] = imp_dti.transform(evaluation[['dti']])

In [50]:
evaluation[['pay_day_test_result_amount']] = imp_pay_day.transform(evaluation[['pay_day_test_result_amount']])

In [52]:
evaluation['LeadProvider'] = evaluation['LeadProvider'].fillna('freedom')

In [53]:
evaluation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1092 entries, 21 to 21565
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   loan_id                     1092 non-null   object        
 1   LoanCount                   1092 non-null   int64         
 2   OriginationDate             1092 non-null   datetime64[ns]
 3   Campaign                    1091 non-null   object        
 4   MonthlyGrossIncome          1092 non-null   float64       
 5   DateOfBirth                 1092 non-null   datetime64[ns]
 6   IsFirstDefault              1092 non-null   bool          
 7   Age                         1092 non-null   int64         
 8   LeadProvider                1092 non-null   object        
 9   Reloan                      1092 non-null   bool          
 10  AccessCount                 1092 non-null   float64       
 11  BankReportData              1092 non-null   object    

In [40]:
import ppscore as pps

In [44]:
new_feat = ['avg_daily_debit', 'avg_daily_credit', 'avg_daily_debit_count',
            'avg_daily_credit_count', 'avg_daily_balance', 'median_daily_debit',
            'median_daily_credit', 'median_daily_debit_count',
            'median_daily_credit_count', 'median_daily_balance', 'dev_daily_debit',
            'dev_daily_credit', 'dev_daily_debit_count', 'dev_daily_credit_count',
            'dev_daily_balance', 'avg_weekly_debit', 'avg_weekly_credit',
            'avg_weekly_debit_count', 'avg_weekly_credit_count',
            'avg_weekly_balance', 'median_weekly_debit', 'median_weekly_credit',
            'median_weekly_debit_count', 'median_weekly_credit_count',
            'median_weekly_balance', 'dev_weekly_debit', 'dev_weekly_credit',
            'dev_weekly_debit_count', 'dev_weekly_credit_count',
            'dev_weekly_balance']

In [79]:
scores = []
for i in new_feat:
    scores.append(pps.score(training, i, 'IsFirstDefault'))

In [80]:
scores

[{'x': 'avg_daily_debit',
  'y': 'IsFirstDefault',
  'task': 'classification',
  'ppscore': 0,
  'metric': 'weighted F1',
  'baseline_score': 0.7250122781872484,
  'model_score': 0.6978836348642454,
  'model': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                         max_depth=None, max_features=None, max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, presort='deprecated',
                         random_state=None, splitter='best')},
 {'x': 'avg_daily_credit',
  'y': 'IsFirstDefault',
  'task': 'classification',
  'ppscore': 0,
  'metric': 'weighted F1',
  'baseline_score': 0.7250122781872484,
  'model_score': 0.6789038268999628,
  'model': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                         max_depth=None, max_features=None, max_l

In [81]:
from scipy.stats import pointbiserialr

In [82]:
corr = {}
for i in new_feat:
    corr[i] = pointbiserialr(training[i], training['IsFirstDefault'])

In [83]:
corr

{'avg_daily_debit': PointbiserialrResult(correlation=0.021625443120991544, pvalue=0.1457266727048597),
 'avg_daily_credit': PointbiserialrResult(correlation=-0.026316229699948627, pvalue=0.07665185726165688),
 'avg_daily_debit_count': PointbiserialrResult(correlation=-0.008331353059804848, pvalue=0.5751976149835424),
 'avg_daily_credit_count': PointbiserialrResult(correlation=-0.011136527757291506, pvalue=0.45378781727706746),
 'avg_daily_balance': PointbiserialrResult(correlation=0.008231476143870904, pvalue=0.57978779258136),
 'median_daily_debit': PointbiserialrResult(correlation=0.039020610526698224, pvalue=0.00864714887732418),
 'median_daily_credit': PointbiserialrResult(correlation=-0.012982034115276065, pvalue=0.3825183359730789),
 'median_daily_debit_count': PointbiserialrResult(correlation=-0.016186285526027352, pvalue=0.27622790202466946),
 'median_daily_credit_count': PointbiserialrResult(correlation=-0.0220142250366479, pvalue=0.1386190504506542),
 'median_daily_balance': 

In [258]:
df_loans = training[['MonthlyGrossIncome', 'Age', 'Reloan', 'LeadProvider', 'LenderCountCred30', 
                     'UniqLenderCount', 'LenderAmountDeb', 'LenderAmountCred', 'LenderAmountDeb30',
                     'LenderAmountCred30', 'LenderCountDeb', 'LenderCountCred', 'LenderCountDeb30',
                     'median_weekly_credit', 'dev_weekly_credit_count', 'median_daily_debit', 'AccessCount', 
                     'IsFirstDefault', 'dti', 'pay_day_test_result_amount', 'diff_pos_neg_days', 'median_daily_balance']]

In [259]:
df_loans = prep.changing_bool_dtypes_to_str(df_loans)

In [260]:
df_loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4527 entries, 1 to 21561
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   MonthlyGrossIncome          4527 non-null   float64
 1   Age                         4527 non-null   int64  
 2   Reloan                      4527 non-null   object 
 3   LeadProvider                4527 non-null   object 
 4   LenderCountCred30           4527 non-null   float64
 5   UniqLenderCount             4527 non-null   float64
 6   LenderAmountDeb             4527 non-null   float64
 7   LenderAmountCred            4527 non-null   float64
 8   LenderAmountDeb30           4527 non-null   float64
 9   LenderAmountCred30          4527 non-null   float64
 10  LenderCountDeb              4527 non-null   float64
 11  LenderCountCred             4527 non-null   float64
 12  LenderCountDeb30            4527 non-null   float64
 13  median_weekly_credit        4527

## Pycaret

In [70]:
from pycaret.classification import *

In [261]:
clf = setup(data = df_loans, train_size = .99, target = 'IsFirstDefault', session_id = 69,
            normalize = True,
            transformation = True,
            remove_outliers = True, 
            numeric_features = ['MonthlyGrossIncome', 'Age', 'LenderCountCred30', 
                                'UniqLenderCount', 'LenderAmountDeb', 'LenderAmountCred', 'LenderAmountDeb30',
                                'LenderAmountCred30', 'LenderCountDeb', 'LenderCountCred', 'LenderCountDeb30',
                                'median_weekly_credit', 'dev_weekly_credit_count', 'median_daily_debit',
                                'AccessCount', 'dti', 'pay_day_test_result_amount', 'diff_pos_neg_days', 'median_daily_balance'])

 
Setup Succesfully Completed!


,Description,Value
0,session_id,69
1,Target Type,Binary
2,Label Encoded,"False: 0, True: 1"
3,Original Data,"(4527, 23)"
4,Missing Values,False
5,Numeric Features,20
6,Categorical Features,2
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [262]:
tuned_lr = tune_model('lr', optimize = 'F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.5352,0.6461,0.6500,0.2342,0.3444,0.0943
1,0.6009,0.6858,0.7250,0.2816,0.4056,0.1852
2,0.5986,0.6934,0.6750,0.2714,0.3871,0.1628
3,0.5610,0.6677,0.7000,0.2557,0.3746,0.1372
4,0.5798,0.6604,0.6375,0.2537,0.3630,0.1290
5,0.5775,0.6799,0.7125,0.2664,0.3878,0.1574
6,0.5812,0.6528,0.6875,0.2644,0.3819,0.1511
7,0.5765,0.6769,0.6875,0.2619,0.3793,0.1467
8,0.5835,0.7043,0.7000,0.2679,0.3875,0.1584
9,0.5647,0.6498,0.6875,0.2558,0.3729,0.1358


In [263]:
bagged_lr = ensemble_model(tuned_lr, method = 'Bagging')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.5423,0.6433,0.6625,0.2398,0.3522,0.1055
1,0.5962,0.6833,0.7000,0.2745,0.3944,0.1706
2,0.6056,0.6928,0.6625,0.2732,0.3869,0.1647
3,0.5728,0.6653,0.6750,0.2571,0.3724,0.1380
4,0.5822,0.6625,0.6375,0.2550,0.3643,0.1312
5,0.5915,0.6848,0.7250,0.2762,0.4000,0.1759
6,0.5671,0.6486,0.6750,0.2547,0.3699,0.1328
7,0.5741,0.6779,0.6750,0.2584,0.3737,0.1394
8,0.5812,0.7011,0.6625,0.2598,0.3732,0.1409
9,0.5741,0.6523,0.7000,0.2629,0.3823,0.1495


In [264]:
final_model = finalize_model(bagged_lr)

In [257]:
save_model(final_model, 'new_model')

Transformation Pipeline and Model Succesfully Saved


## Evaluation

In [273]:
df_eval = evaluation[['MonthlyGrossIncome', 'Age', 'Reloan', 'LeadProvider', 'LenderCountCred30', 
                     'UniqLenderCount', 'LenderAmountDeb', 'LenderAmountCred', 'LenderAmountDeb30',
                     'LenderAmountCred30', 'LenderCountDeb', 'LenderCountCred', 'LenderCountDeb30',
                     'median_weekly_credit', 'dev_weekly_credit_count', 'median_daily_debit', 'AccessCount', 
                     'IsFirstDefault', 'dti', 'pay_day_test_result_amount', 'diff_pos_neg_days', 'median_daily_balance']]

In [274]:
df_eval = prep.changing_bool_dtypes_to_str(df_eval)

In [277]:
new_model = load_model('new_model')

Transformation Pipeline and Model Sucessfully Loaded


In [278]:
predictions_pycaret = predict_model(new_model, data = df_eval)

In [279]:
eval_.get_KS(predictions_pycaret, 'IsFirstDefault', 'Score')

0.2307836872768277